<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/04%20-%20Neural%20Network%20Basic/01_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

포유류인지 조류인지 분류하는 간단한 신경망 모델을 구현

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

## 데이터 정의

In [ ]:
# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
# 다음과 같은 형식으로 분류를 하는 것을 one-hot 형식 레이블(분류값)이라고 한다.
# 해당하는 종류만 1을 가지고 나머지는 모두 0인 인덱스로 표현한다.
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

## 신경망 모델 구성

### 변수와 가중치,편향 정의

In [ ]:
# x,y 플레이스 홀더 정의
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망은 [2, 3]의 크기를 가지며 -1 ~ 1 사이의 랜덤한 값을 넣도록 한다.
# [입력층(털, 날개 2개), 출력층(기타, 포유류, 조류 3개)] -> [2, 3] 이기 때문이다.
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))

# 편향은 각 레이어의 출력수로 설정한다.
# 여기에서는 최종 결과값의 분류 갯수인 3이 출력수로 설정된다.
b = tf.Variable(tf.zeros([3]))

### 신경망 구성과 활성화 함수

In [ ]:
# 신경망에 가중치 W과 편향 b을 적용한다.
L = tf.add(tf.matmul(X, W), b)
# 가중치와 편향을 이용해 계산한 결과 값에
# 텐서플로우에서 기본적으로 제공하는 활성화 함수인 ReLU 함수를 적용한다.
L = tf.nn.relu(L)

# 결과값을 전체합이 1인 확률로 만들어주는 활성화 함수인 softmax를 사용하여
# 출력값을 보기 쉽게 만든다.
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

### 비용 함수

In [ ]:
# 신경망을 최적화하기 위한 비용 함수를 작성한다.각 개별 결과에 대한 합을 구한 뒤 평균을 내어 비용을 구한다
# 전체 합이 아닌, 각각의 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션을 사용한다.
# axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력된다.

#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)   reduce_mean
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]  ->    -0.545
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]

# 위의 reduce_mean은 (예측값과 실제값 사이의 확률 분포의 차이를 계산한 것으로) Cross-Entropy 라고 한다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

### 최적화 함수

In [ ]:
# 최적화 방식은 경사하강법으로 한다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

## 모델 학습 및 테스트

### 모델 학습

In [ ]:
# 신경망 모델 학습

# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 위에서 구성한 특징과 레이블로 100번의 학습을 진행한다.
for step in range(100):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  # 10번마다 손실값을 출력하도록 한다.
  if (step + 1) % 10 == 0:
    print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.3313376
20 1.3181595
30 1.3053194
40 1.2928058
50 1.2806076
60 1.2687138
70 1.2571145
80 1.2458
90 1.2347614
100 1.2268143


### 모델 테스트

In [ ]:
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류

# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가지는 *인덱스*를 가져온다.
# 위의 함수를 사용하여 one-hot 인코딩에서 레이블을 정보를 가져올 수 있다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# tf.equal : 두 값이 같으면 True 아니면 False를 반환
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 0 0 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 50.00


# 전체 코드

In [ ]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#########
# 신경망 모델 구성
######

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

model = tf.nn.softmax(L)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


#########
# 신경망 모델 학습
######

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))